## **Laboratorio 9**
## **Julio García Salas - 22076**
## **Sofía García - 22210**

# Modelos de Markov y Hidden Markov Models (HMM)

## 1. Diferencia entre Modelos de Markov y Hidden Markov Models

Un **Modelo de Markov (Markov Chain)** es un modelo probabilístico que describe una secuencia de posibles eventos en los que la probabilidad de cada evento depende únicamente del estado alcanzado en el evento anterior. Es decir, el proceso tiene la propiedad de **Markov**, también conocida como **memoria limitada**.

- En un Modelo de Markov clásico, los **estados son observables**.
- Se describe con una matriz de transición $A$ entre estados.

Un **Modelo Oculto de Markov (Hidden Markov Model, HMM)** es una extensión del modelo de Markov donde el **estado real no es observable directamente** (es "oculto"), pero se puede inferir a partir de **observaciones** que dependen probabilísticamente de los estados ocultos.

- En un HMM, se tiene:
  - Una secuencia de **estados ocultos** $q_1, q_2, ..., q_T$
  - Una secuencia de **observaciones** $o_1, o_2, ..., o_T$
  - Probabilidades de emisión $B$ que definen la probabilidad de observar $o_t$ dado el estado oculto $q_t$

## 2. ¿Qué son los factorial HMM?

Los **Factorial Hidden Markov Models (FHMM)** son una generalización de los HMM tradicionales. En lugar de tener una única cadena de estados ocultos, un FHMM utiliza **varias cadenas ocultas que evolucionan en paralelo** y que juntas determinan la distribución de las observaciones.

- Cada cadena de estados ocultos sigue un modelo de Markov independiente.
- Las observaciones se generan a partir de la **combinación de múltiples cadenas ocultas**.

Esto permite modelar **dependencias más complejas** entre los estados y las observaciones. Son útiles en contextos donde múltiples factores ocultos influyen en los datos observados (por ejemplo, reconocimiento de actividad humana, bioinformática, etc.).

## 3. Algoritmo Forward-Backward para HMM

El algoritmo **Forward-Backward** es una técnica de inferencia utilizada en HMMs para **calcular la probabilidad de una secuencia de observaciones** y para inferir la **probabilidad posterior de los estados ocultos** en cada tiempo dado.

Se divide en dos fases:

- **Forward ($\alpha$) paso**: calcula de manera recursiva la probabilidad de la secuencia observada hasta el tiempo $t$ y de estar en un estado $i$ en ese momento:
  
  $$
  \alpha_t(i) = P(o_1, o_2, ..., o_t, q_t = i \mid \lambda)
  $$

- **Backward ($\beta$) paso**: calcula la probabilidad de observar el resto de la secuencia desde $t+1$ en adelante, dado que el sistema está en el estado $i$ en el tiempo $t$:

  $$
  \beta_t(i) = P(o_{t+1}, o_{t+2}, ..., o_T \mid q_t = i, \lambda)
  $$

- Finalmente, la probabilidad posterior de estar en un estado $i$ en el tiempo $t$ dado la secuencia completa de observaciones es:

  $$
  \gamma_t(i) = \frac{\alpha_t(i) \cdot \beta_t(i)}{\sum_{j=1}^N \alpha_t(j) \cdot \beta_t(j)}
  $$

## 4. ¿Por qué es necesario el paso Backward?

El paso **Backward** permite incorporar la información **futura** (las observaciones posteriores al tiempo $t$) para estimar correctamente la probabilidad de que el sistema haya estado en un determinado estado en ese tiempo.

### Ejemplo:

Supongamos que tenemos una observación inesperada al final de la secuencia que difícilmente pudo haber ocurrido en algunos estados anteriores. Sin el paso **Backward**, el algoritmo no tendría forma de ajustar las probabilidades anteriores con base en esa nueva información.

- Por ejemplo, si en el tiempo $t=1$ se considera probable estar en el estado $S_1$, pero en el tiempo $t=3$ se observa un evento muy improbable desde $S_1$, el paso backward puede ajustar hacia abajo la probabilidad de $S_1$ en $t=1$.

Esto se debe a que los **HMM son modelos generativos de toda la secuencia**, y no solo del presente. Sin el paso backward, solo se tendría una visión parcial (solo hacia adelante) y se perdería contexto importante que afecta la inferencia.


## Task 2

In [ ]:
import random

class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    def generate_sequence(self, length):
        sequence = []
        current_state = random.choices(self.states, weights=[self.initial_prob[s] for s in self.states])[0]
        for _ in range(length):
            obs = random.choices(self.observations, weights=[self.emission_prob[current_state][o] for o in self.observations])[0]
            sequence.append(obs)
            current_state = random.choices(self.states, weights=[self.transition_prob[current_state][s] for s in self.states])[0]
        return sequence

    def forward(self, observations):
        alpha = []
        for t, obs in enumerate(observations):
            alpha_t = {}
            for state in self.states:
                if t == 0:
                    alpha_t[state] = self.initial_prob[state] * self.emission_prob[state][obs]
                else:
                    alpha_t[state] = sum(
                        alpha[t-1][prev_state] * self.transition_prob[prev_state][state] for prev_state in self.states
                    ) * self.emission_prob[state][obs]
            alpha.append(alpha_t)
        return alpha

    def backward(self, observations):
        beta = [{} for _ in observations]
        T = len(observations)
        for state in self.states:
            beta[T-1][state] = 1
        for t in reversed(range(T-1)):
            for state in self.states:
                beta[t][state] = sum(
                    self.transition_prob[state][next_state] *
                    self.emission_prob[next_state][observations[t+1]] *
                    beta[t+1][next_state]
                    for next_state in self.states
                )
        return beta

    def compute_state_probabilities(self, observations):
        alpha = self.forward(observations)
        beta = self.backward(observations)
        gamma = []
        for t in range(len(observations)):
            prob_sum = sum(alpha[t][s] * beta[t][s] for s in self.states)
            gamma_t = {s: (alpha[t][s] * beta[t][s]) / prob_sum for s in self.states}
            gamma.append(gamma_t)
        return gamma


Secuencia Generada: None
Probabilidades Forward:
None
Probabilidades Backward:
None
Probabilidades de Estados:
None
